In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import requests
from tqdm import trange

## 读入处理重复数据，划分测试集和训练集

In [2]:
data = pd.read_csv("tobacco.csv")
data

,area,url,yellow,water,black,temperature,humidity
0,1,http://218.94.8.117:9037/adp/pingdingshan/01sh...,0.0,0.0,NaN,1.0,2.0
1,1,http://218.94.8.117:9037/adp/pingdingshan/01sh...,0.0,0.0,NaN,1.0,2.0
2,1,http://218.94.8.117:9037/adp/pingdingshan/01sh...,0.0,0.0,NaN,1.0,2.0
3,1,http://218.94.8.117:9037/adp/pingdingshan/01sh...,0.0,1.0,NaN,2.0,2.0
4,1,http://218.94.8.117:9037/adp/pingdingshan/01sh...,0.0,1.0,NaN,2.0,2.0
...,...,...,...,...,...,...,...
9217,5,http://218.94.8.117:9037/adp/nanyang/nanyang04...,4.0,6.0,NaN,8.0,2.0
9218,5,http://218.94.8.117:9037/adp/nanyang/nanyang04...,5.0,6.0,NaN,8.0,2.0
9219,5,http://218.94.8.117:9037/adp/nanyang/nanyang04...,5.0,7.0,NaN,9.0,1.0
9220,5,http://218.94.8.117:9037/adp/nanyang/nanyang04...,6.0,7.0,NaN,9.0,1.0


In [19]:
data.groupby('url').get_group("http://218.94.8.117:9037/adp/pingdingshan/01shangpeng/01shangpeng120220920190124.jpg")

,area,url,yellow,water,black,temperature,humidity
39,1,http://218.94.8.117:9037/adp/pingdingshan/01sh...,5.0,7.0,NaN,NaN,NaN
40,1,http://218.94.8.117:9037/adp/pingdingshan/01sh...,NaN,NaN,NaN,8.0,2.0


In [17]:
# 观察重复数目
dup_filter = data.duplicated(subset=['url'],keep="first").sum()
dup_filter

1738

In [4]:
# 合并数据集同一图片的重复行，输出到本地
# 所有同一图片的重复行，包含图片的不同指标，所以必须合并行
url_group = data.groupby('url')
data_nodup = url_group[["area","yellow",'water',"black","temperature","humidity"]].mean().reset_index() \
                        .sort_values(by=["area","url"],ascending=[True,True]).reset_index().drop(columns="index")
data_nohup_path = Path("tobacco_nohuop.csv")
if not data_nohup_path.exists():
    print(f"downloading {data_nohup_path}..")
    data_nodup.to_csv(data_nohup_path)
else:
    print(f"{data_nohup_path} exists !")

tobacco_nohuop.csv exists !


In [5]:
# 划分训练集测试集
train_test_filt = (data_nodup['area'] == 5)
data_test = data_nodup[train_test_filt]
data_train = data_nodup[~train_test_filt]
data_train

,url,area,yellow,water,black,temperature,humidity
0,http://218.94.8.117:9037/adp/pingdingshan/01sh...,1.0,0.0,0.0,NaN,1.0,2.0
1,http://218.94.8.117:9037/adp/pingdingshan/01sh...,1.0,0.0,0.0,NaN,1.0,2.0
2,http://218.94.8.117:9037/adp/pingdingshan/01sh...,1.0,0.0,0.0,NaN,1.0,2.0
3,http://218.94.8.117:9037/adp/pingdingshan/01sh...,1.0,0.0,1.0,NaN,2.0,2.0
4,http://218.94.8.117:9037/adp/pingdingshan/01sh...,1.0,0.0,1.0,NaN,2.0,2.0
...,...,...,...,...,...,...,...
6877,http://218.94.8.117:9037/adp/sanmenxia11/sanme...,4.0,6.0,8.0,NaN,NaN,NaN
6878,http://218.94.8.117:9037/adp/sanmenxia11/sanme...,4.0,6.0,8.0,NaN,NaN,NaN
6879,http://218.94.8.117:9037/adp/sanmenxia11/sanme...,4.0,6.0,8.0,NaN,NaN,NaN
6880,http://218.94.8.117:9037/adp/sanmenxia11/sanme...,4.0,6.0,8.0,NaN,9.0,2.0


## 探测数据情况

In [6]:
data_train.describe()

,area,yellow,water,black,temperature,humidity
count,6882.000000,6456.000000,6469.000000,1928.000000,5314.000000,6035.000000
mean,1.836821,2.707559,3.637966,2.263485,4.922093,1.986578
std,0.927369,1.937276,2.258363,1.374040,2.618462,0.265679
min,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,2.000000,1.000000,2.000000,2.000000
50%,2.000000,3.000000,3.000000,2.000000,4.000000,2.000000
75%,2.000000,4.000000,5.000000,3.000000,7.000000,2.000000
max,4.000000,6.000000,8.000000,5.000000,9.000000,3.000000


In [7]:
data_test.describe()

,area,yellow,water,black,temperature,humidity
count,602.0,594.000000,592.000000,110.000000,202.000000,203.000000
mean,5.0,2.809764,3.998311,1.172727,4.811881,1.985222
std,0.0,2.027427,2.497037,0.379741,2.549853,0.120963
min,5.0,0.000000,0.000000,1.000000,1.000000,1.000000
25%,5.0,1.000000,2.000000,1.000000,2.000000,2.000000
50%,5.0,3.000000,4.000000,1.000000,5.000000,2.000000
75%,5.0,4.000000,6.000000,1.000000,7.000000,2.000000
max,5.0,6.000000,8.000000,2.000000,9.000000,2.000000


In [8]:
data_train["water"].value_counts()

water
2.0    1581
5.0     867
1.0     768
3.0     692
7.0     685
4.0     661
6.0     547
0.0     359
8.0     309
Name: count, dtype: int64

In [9]:
data_test["water"].value_counts()

water
2.0    105
6.0     97
1.0     70
8.0     68
5.0     64
4.0     57
3.0     49
0.0     44
7.0     38
Name: count, dtype: int64

In [10]:
# 统计标签个数,并按照标签进一步划分训练/测试集
water_label_num = data_nodup["water"].nunique(dropna=True)
def split_data(dataset,label_num,label_name:str,set_type:str = "train") -> None:
    """将训练集/测试集按照标签进一步划分，并且输出为csv文件"""
    for label in range(label_num):
        set_path = Path(f"{label_name}_{set_type}_{label}.csv")
        if not set_path.exists():
            print(f"split label:{set_path.stem}...")
            label_filter = dataset[label_name] == label
            exec(f"{label_name}_{set_type}_{label} = data_{set_type}.loc[label_filter]")
            exec(f"{label_name}_{set_type}_{label}.to_csv(f'{set_path}')")
        else:
            print(f"{set_path} exists...")
            
split_data(data_train,water_label_num,"water","train")
split_data(data_test,water_label_num,"water","test")

water_train_0.csv exists...
water_train_1.csv exists...
water_train_2.csv exists...
water_train_3.csv exists...
water_train_4.csv exists...
water_train_5.csv exists...
water_train_6.csv exists...
water_train_7.csv exists...
water_train_8.csv exists...
water_test_0.csv exists...
water_test_1.csv exists...
water_test_2.csv exists...
water_test_3.csv exists...
water_test_4.csv exists...
water_test_5.csv exists...
water_test_6.csv exists...
water_test_7.csv exists...
water_test_8.csv exists...


In [15]:
def download_all(csv_list:list[str],set_type:str) -> None: 
    def download_data(root_list:list[str],destination_dir:str):
        destination_dir = Path(destination_dir)
        if not destination_dir.exists():
            destination_dir.mkdir(exist_ok=True,parents=True)
            print(f"create dir: {destination_dir}")
        pic_num = 1
        root_list = sorted(root_list)
        for root in root_list:
            dest = destination_dir / f"{pic_num}.jpg"
            if dest.exists():
                print(f"pic {pic_num} existed")
            else:
                with open(dest,"wb") as wf:
                    r = requests.get(root)
                    wf.write(r.content)
                    print(f"has downloaded pic : {pic_num}")
            pic_num += 1
        print(f"downloaded {pic_num-1} pictures in total")
    
    label = 0 
    for csv in csv_list:
        data = pd.read_csv(csv)
        root_list = data['url'].tolist()
        destination_dir = f"img_data/{set_type}/{label}"
        download_data(root_list,destination_dir)
        print(f"has downloaded {set_type} label set : {label}")
        label += 1
    print(f"has downloaded {label} {set_type} label sets in total")

test_csvlist = [f"water_test_{i}.csv" for i in range(9)]
train_csvlist = [f"water_train_{i}.csv" for i in range(9)]
download_all(test_csvlist,"test")
download_all(train_csvlist,"train")

pic 1 existed
pic 2 existed
pic 3 existed
pic 4 existed
pic 5 existed
pic 6 existed
pic 7 existed
pic 8 existed
pic 9 existed
pic 10 existed
pic 11 existed
pic 12 existed
pic 13 existed
pic 14 existed
pic 15 existed
pic 16 existed
pic 17 existed
pic 18 existed
pic 19 existed
pic 20 existed
pic 21 existed
pic 22 existed
pic 23 existed
pic 24 existed
pic 25 existed
pic 26 existed
pic 27 existed
pic 28 existed
pic 29 existed
pic 30 existed
pic 31 existed
pic 32 existed
pic 33 existed
pic 34 existed
pic 35 existed
pic 36 existed
pic 37 existed
pic 38 existed
pic 39 existed
pic 40 existed
pic 41 existed
pic 42 existed
pic 43 existed
pic 44 existed
downloaded 44 pictures in total
has downloaded test label set : 0
pic 1 existed
pic 2 existed
pic 3 existed
pic 4 existed
pic 5 existed
pic 6 existed
pic 7 existed
pic 8 existed
pic 9 existed
pic 10 existed
pic 11 existed
pic 12 existed
pic 13 existed
pic 14 existed
pic 15 existed
pic 16 existed
pic 17 existed
pic 18 existed
pic 19 existed
pic 20 

In [3]:
%load_ext tensorboard
%tensorboard --logdir="runs" --port=8866

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 8866 (pid 43740), started 0:00:53 ago. (Use '!kill 43740' to kill it.)